Reference: [https://www.kaggle.com/yamqwe/crypto-prediction-xgb-regressor](https://www.kaggle.com/yamqwe/crypto-prediction-xgb-regressor)

In [1]:
import traceback
import numpy as np
import pandas as pd
import datatable as dt
from lightgbm import LGBMRegressor
import xgboost as xgb

TRAIN_JAY = './data/cryptocurrency-extra-data-binance-coin/orig_train.jay'
ASSET_DETAILS_JAY = './data/cryptocurrency-extra-data-binance-coin/orig_asset_details.jay'

In [2]:
df_train = dt.fread('./data/cryptocurrency-extra-data-binance-coin/orig_train.jay').to_pandas()
df_asset_details = dt.fread('./data/cryptocurrency-extra-data-binance-coin/orig_asset_details.jay').to_pandas().sort_values("Asset_ID")

## Utility functions to train a model for one asset

**Main Training Function**, **Feature Extraction**

In [3]:
from script.XGB.xgbmodel import *

## Loop over all assets

In [4]:
Xs = {}
ys = {}
models = {}

for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    try:
        X, y, model = get_Xy_and_model_for_asset(df_train, asset_id)    
        Xs[asset_id], ys[asset_id], models[asset_id] = X, y, model
    except:         
        Xs[asset_id], ys[asset_id], models[asset_id] = None, None, None    
    #save model
    models[asset_id].save_model(f"./trainedXGB/model_{asset_id}.json")

Training model for Binance Coin     (ID=0 )


In [ ]:
# Check the model interface
x = get_features(df_train.iloc[1])
y_pred = models[0].predict(pd.DataFrame([x]))
y_pred[0]

# Submit To Kaggle

In [ ]:
'''
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for i, (df_test, df_pred) in enumerate(iter_test):
    for j , row in df_test.iterrows():
        
        if models[row['Asset_ID']] is not None:
            try:
                model = models[row['Asset_ID']]
                x_test = get_features(row)
                y_pred = model.predict(pd.DataFrame([x_test]))[0]
                df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
            except:
                df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
                traceback.print_exc()
        else: 
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
        
    env.predict(df_pred)
'''